<font color='darkred'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Exercises

For these exercises, add your functions to the *apputil\.py* file. If you like, you're welcome to adjust the *app\.py* file, but it is not required.

## Notes on Recursion

A [recursive function](https://www.w3schools.com/python/gloss_python_function_recursion.asp) is one which calls itself.

1. When the function is called, your CPU runs through each line of code until the function needs to be called again.
2. At that point, all variables are saved in memory, and the function runs through each line of code again until the function is called (again, but with a different passed argument), and so on.
3. Eventually, this process will stop at the "bottom of the **stack**", where the function doesn't get a chance to call itself again (likely because of some condition un/met by the latest passed argument).
4. Then, your CPU will work its way back up the stack to the final result. For example, take a look at [this visual example](https://realpython.com/python-recursion/#calculate-factorial) of calculating 4!.

When you write these functions, keep two things in mind:

- You will need a built-in stopping point (i.e., the "bottom"), where your function returns some result before it calls itself.
- **Don't think too hard about this.** Recursion can be perplexing to conceptualize when writing the code. So, when you call the function inside the function, think about it as a magical "hidden" function that has already done what you want it to do.
- [Python Tutor](https://pythontutor.com/) ([editor](https://pythontutor.com/visualize.html#mode=edit)) can be a helpful resource for this exercise!

## Exercise 1

The Fibonacci Series starts with 0 and 1. Each of the following numbers are the sum of the previous two numbers in the series:

`0 1 1 2 3 5 8 13 21 34 ...`

So, `fib(9) = 34`.

Write a recursive function (`fib`) that, given `n`, will return the `n`th number of the Fibonacci Series.

*Test your function using Google or any other tool that can calculate the Fibonacci Series.*

In [11]:
def fib(n: int) -> int:
    if not isinstance(n, int):
        raise ValueError("n must be an integer")
    if n < 0:
        raise ValueError("n must be >= 0")

    if n in (0, 1):
        return n

    return fib(n - 1) + fib(n - 2)


fib(9)


34


## Exercise 2

Write a (single) recursive function, `to_binary()`, that [converts](https://en.wikipedia.org/wiki/Binary_number#Conversion_to_and_from_other_numeral_systems) an integer into its [binary](https://en.wikipedia.org/wiki/Binary_number) representation. So, for example:

```python
to_binary(2)   -->  10
to_binary(12)  -->  1100
```

*Note: you can test your function with the built in `bin()` function.*

In [12]:
def to_binary(n: int) -> str:
    """Return the binary (base-2) representation of a non-negative integer.

    Uses a single recursive function. For example:
        to_binary(2)  -> "10"
        to_binary(12) -> "1100"

    Args:
        n: Integer to convert. Must be >= 0.

    Returns:
        Binary string without the '0b' prefix.

    Raises:
        ValueError: If n is negative or not an integer.
    """
    if not isinstance(n, int):
        raise ValueError("n must be an integer")
    if n < 0:
        raise ValueError("n must be >= 0")

    if n in (0, 1):
        return str(n)

    return to_binary(n // 2) + str(n % 2)

to_binary(2)

'10'

## Exercise 3 

Use the raw Bellevue Almshouse Dataset (`df_bellevue`) extracted at the top of the lab (i.e., with `pd.read_csv ...`).

**Write a function for each of the following tasks. Name these functions `task_i()`** (i.e., without any input arguments).

1. Return a list of all column names, *sorted* such that the first column has the *least* missing values, and the last column has the *most* missing values (use the raw column names).
   - *Note: there is an issue with the `gender` column you'll need to remedy first ...*
2. Return a **data frame** with two columns:
   - the year (for each year in the data), `year`
   - the total number of entries (immigrant admissions) for each year, `total_admissions`
3. Return a **series** with:
   - Index: gender (for each gender in the data)
   - Values: the average age for the indexed gender.
4. Return a list of the 5 most common professions *in order of prevalence* (so, the most common is first).

For each of these, if there are messy data issues, use the `print` statement to explain.


In [18]:
import pandas as pd


def task_1():
    """Column names sorted by fewest -> most missing values."""
    df = df_bellevue.copy()

    if "gender" in df.columns:
        df["gender"] = (
            df["gender"].astype("string").str.strip().str.lower()
            .replace({"": pd.NA, "na": pd.NA, "n/a": pd.NA, "none": pd.NA})
            .replace({
                "m": "male", "h": "male", "b": "male", "boy": "male", "man": "male",
                "f": "female", "w": "female", "g": "female", "girl": "female", "woman": "female"
            })
        )

    return df.isna().sum().sort_values().index.tolist()


def task_2():
    """DataFrame with 'year' and 'total_admissions'."""
    df = df_bellevue.copy()

    if "year" in df.columns:
        year = pd.to_numeric(df["year"], errors="coerce")
    elif "date_in" in df.columns:
        year = pd.to_datetime(df["date_in"], errors="coerce").dt.year
    else:
        print("Need 'year' or 'date_in'.")
        return pd.DataFrame({"year": [], "total_admissions": []})

    out = (
        year.dropna().astype(int).to_frame("year")
        .groupby("year").size().reset_index(name="total_admissions")
        .sort_values("year").reset_index(drop=True)
    )
    return out


def task_3():
    """Series: average age by gender."""
    df = df_bellevue.copy()

    if "gender" not in df.columns or "age" not in df.columns:
        print("Need 'gender' and 'age'.")
        return pd.Series(dtype="float64", name="age")

    gender = (
        df["gender"].astype("string").str.strip().str.lower()
        .replace({"": pd.NA, "na": pd.NA, "n/a": pd.NA, "none": pd.NA})
        .replace({
            "m": "male", "h": "male", "b": "male", "boy": "male", "man": "male",
            "f": "female", "w": "female", "g": "female", "girl": "female", "woman": "female"
        })
    )
    age = pd.to_numeric(df["age"], errors="coerce")

    avg = (
        pd.DataFrame({"gender": gender, "age": age})
        .dropna().groupby("gender")["age"].mean().round(6)
    )
    return avg


def task_4():
    """Top 5 professions (most common first)."""
    df = df_bellevue.copy()

    col = "profession"
    if col not in df.columns:
        if "occupation" in df.columns:
            col = "occupation"
        else:
            print("No profession/occupation column.")
            return []

    s = df[col].astype("string").str.strip().str.lower().replace({"": pd.NA}).dropna()
    return s.value_counts().head(5).index.tolist()


In [19]:
import pandas as pd
# load your CSV once (adjust path)

url = 'https://github.com/melaniewalsh/Intro-Cultural-Analytics/raw/master/book/data/bellevue_almshouse_modified.csv'

df_bellevue = pd.read_csv(url)
# df_bellevue = pd.read_csv('./data/.../mydata.csv')  # you can also reference locally stored data

# now call the functions you defined
print("Task 1 (cols by missingness):")
print(task_1())

print("\nTask 2 (year + total_admissions):")
print(task_2().head())  # .head() so it’s short

print("\nTask 3 (avg age by gender):")
print(task_3())

print("\nTask 4 (top 5 professions):")
print(task_4())


Task 1 (cols by missingness):
['date_in', 'last_name', 'gender', 'first_name', 'age', 'profession', 'disease', 'children']

Task 2 (year + total_admissions):
   year  total_admissions
0  1846              3073
1  1847              6511

Task 3 (avg age by gender):
gender
female    28.738328
male      31.818334
Name: age, dtype: float64

Task 4 (top 5 professions):
['laborer', 'married', 'spinster', 'widow', 'shoemaker']
